In [4]:
%load_ext autoreload
%autoreload 2
import torch

from sae.sparse_autoencoder import load_saved_sae
from sae.metrics import model_store_from_sae
from unlearning.metrics import convert_wmdp_data_to_prompt, calculate_MCQ_metrics, all_permutations
from unlearning.tool import UnlearningConfig, SAEUnlearningTool, MCQ_ActivationStoreAnalysis

from huggingface_hub import hf_hub_download
from datasets import load_dataset
import numpy as np

from jaxtyping import Float, Int
from torch import Tensor

import plotly.express as px

from transformer_lens import HookedTransformer
from dataclasses import dataclass
import wandb
import einops
from tqdm import tqdm
import json

from functools import partial
from unlearning.intervention import anthropic_remove_resid_SAE_features, remove_resid_SAE_features, anthropic_clamp_resid_SAE_features


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# resid pre 9
REPO_ID = "eoinf/unlearning_saes"
FILENAME = "jolly-dream-40/sparse_autoencoder_gemma-2b-it_blocks.9.hook_resid_pre_s16384_127995904.pt"


filename = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
sae = load_saved_sae(filename)

model = model_store_from_sae(sae)

model-00001-of-00002.safetensors:  43%|####3     | 2.15G/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/34.2k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

Loaded pretrained model gemma-2b-it into HookedTransformer
Moving model to device:  cuda


In [5]:

metrics = calculate_MCQ_metrics(model, 'wmdp-bio')

Generating test split:   0%|          | 0/1273 [00:00<?, ? examples/s]

100%|██████████| 213/213 [01:05<00:00,  3.24it/s]


In [6]:
print(metrics['is_correct'].shape)
print(metrics['mean_correct'])


(1273,)
0.4658287465572357


In [8]:
def modify_model(model, **kwargs):

    default_modification_kwargs = {
        'multiplier': 1.0,
        'intervention_method': 'scale_feature_activation',
        'custom_hook_point': None,
    }
    
    model.reset_hooks()
    
    # Calculate modified stats
    if kwargs['intervention_method'] == "scale_feature_activation":
        ablation_method = anthropic_remove_resid_SAE_features
    elif kwargs['intervention_method'] == "remove_from_residual_stream":
        ablation_method = remove_resid_SAE_features
    elif kwargs['intervention_method'] == "clamp_feature_activation":
        ablation_method = anthropic_clamp_resid_SAE_features
        
    ablate_hook_func = partial(
        ablation_method, 
        sae=sae, 
        features_to_ablate=kwargs['features_to_ablate'],
        multiplier=kwargs['multiplier']
        )
    
    if 'custom_hook_point' not in kwargs or kwargs['custom_hook_point'] is None:
        hook_point = sae.cfg.hook_point
    else:
        hook_point = kwargs['custom_hook_point']
    
    model.add_hook(hook_point, ablate_hook_func)

In [19]:
# modified the model
filtered_good_features = [12663, 4342, 5749, 10355, 1523, 15858, 12273, 14315, 4451, 1611, 10051, 16186, 7983, 6958, 1307, 11019, 6531, 12289]

ablate_params = {
    'features_to_ablate': top_features_from_forget_set[:32],
    'multiplier': 20,
    'intervention_method': 'clamp_feature_activation',
}

modify_model(model, **ablate_params) 

modified_metrics = calculate_MCQ_metrics(model, 'wmdp-bio')
    
model.reset_hooks()

100%|██████████| 213/213 [01:08<00:00,  3.12it/s]


In [21]:
modified_metrics

{'mean_correct': 0.32521602511405945,
 'total_correct': 414,
 'is_correct': array([0., 1., 0., ..., 1., 1., 0.], dtype=float32),
 'output_probs': array([[3.9242983e-01, 4.1326389e-02, 5.0947088e-01, 4.6851005e-02],
        [1.8110876e-01, 1.8788504e-03, 3.1302667e-03, 8.0971557e-01],
        [9.4464011e-02, 1.7328572e-03, 1.1314076e-02, 8.7782061e-01],
        ...,
        [5.6156132e-02, 7.3655997e-04, 9.4066828e-01, 1.1112447e-03],
        [1.6675612e-02, 3.9314985e-02, 9.1423547e-01, 1.7998165e-02],
        [6.3186446e-03, 3.0064736e-03, 3.1196144e-03, 9.8394865e-01]],
       dtype=float32),
 'actual_answers': array([0, 3, 2, ..., 2, 2, 2]),
 'predicted_answers': array([2, 3, 3, ..., 2, 2, 3]),
 'predicted_probs': array([0.5094709 , 0.80971557, 0.8778206 , ..., 0.9406683 , 0.9142355 ,
        0.98394865], dtype=float32),
 'predicted_probs_of_correct_answers': array([0.39242983, 0.80971557, 0.01131408, ..., 0.9406683 , 0.9142355 ,
        0.00311961], dtype=float32),
 'mean_predicted

In [20]:
print(modified_metrics['is_correct'].shape)
print(modified_metrics['mean_correct'])

# 18 features (manually selected) from 60 questions
# 0.2545168995857239

# 8 features from forget set (scale)
# 0.36999213695526123

# 16 features from forget set (scale)
# 0.34878242015838623

# 32 features from forget set (scale)
# 0.3267871141433716
# 0.32521602511405945

(1273,)
0.32521602511405945


In [11]:
top_features_from_forget_set = np.loadtxt('./unlearning_output/top_features_from_forget_set.txt', dtype=int)

In [12]:
top_features_from_forget_set

array([ 1557, 12273,  4271, ..., 11697,  4863, 10364])